# 1. Install optimization model package

In [ ]:
!pip install -q pyomo
# solvers needed to be installed separately

# glpk
!apt-get install -y -qq glpk-utils

!pip install geopy

!pip install gurobipy  # install gurobipy, if not already installed
import gurobipy as gp  # import the installed package


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# check for license expiration date
model_size_limited = gp.Model()

Restricted license - for non-production use only - expires 2024-10-28


In [ ]:
import pandas as pd
import numpy as np
from geopy import distance

# 2. Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/Capstone-KPMG/Preprocessing
!ls

main = pd.read_csv('Data/gas_station_w_dist_to_exit_wholeI5I90I82.csv')
main.columns

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/Capstone-KPMG/Preprocessing
Data


Index(['Unnamed: 0', 'gas_key', 'gas_lat', 'gas_long', 'exit_name', 'exit_lat',
       'exit_long', 'distance_to_nearest_exit', 'interstate'],
      dtype='object')

# 3. Optimization
*Objective function:* Minimize number of chargers/stations

*Constraints:*
- Distance to highway exit < 5 miles
- **All traffic covered (# EV stations * ratio < traffic around).**
- NRI risk rating is not Very High

Electric vehicles were 1.3% of all passenger vehicles on Washington roads

In [ ]:
df = main[main['interstate'].notna()]
df.shape

(910, 9)

In [ ]:
main_with_ntr = pd.read_csv('Data/main_dataset_all_interstate.csv')
main_with_ntr.columns

Index(['gas_key', 'gas_name', 'gas_lat', 'gas_long', 'attr_cnt_1mile',
       'attr_cnt_5mile', 'attr_name', 'attr_lat', 'attr_long',
       'distance_to_nearest_attr', 'crime_coord', 'crime_county',
       'crime_population', 'violent_crime', 'murder_nonnegligent_manslaughter',
       'Rape1', 'Robbery', 'aggravated_assault', 'property_crime', 'Burglary',
       'larceny_theft', 'motor_vehicle_theft', 'Arson', 'total_crime',
       'exit_name', 'exit_lat', 'exit_long', 'distance_to_nearest_exit',
       'exit', 'num_EV_in_2_miles_of_gas', 'num_EV_in_5_miles_of_gas',
       'num_EV_in_10_miles_of_gas', 'num_EV_in_20_miles_of_gas',
       'num_EV_in_50_miles_of_gas', 'Closest_EV_Station_name',
       'Closest_EV_Station_lat', 'Closest_EV_Station_long',
       'distance_to_closest_ev_station', 'nri_geoid', 'nri_county',
       'nri_population', 'nri_build_value', 'nri_agri_value', 'nri_area',
       'nri_risk_score', 'nri_risk_rating', 'nri_intpt_lat', 'nri_intpt_long',
       'nri_zipco

In [ ]:
# filter out fixed constraint data
highways = ['I5', 'I82','I90']
df = main[main['interstate'].isin(highways)]
df_x = df[df['distance_to_nearest_exit'] < 1]
df_x = df_x[df_x['nri_risk_rating']!= 'Very High']
df_x.shape

KeyError: ignored

In [ ]:
def get_adjacenct_gas(candidates):
  # candidates = df_x['gas_key']
  adj = dict()
  for i in range(len(candidates)-1):
      for j in range(i+1, len(candidates)):
        # print(i,j)
        coords_1 = (main.iloc[candidates.iloc[i]]['gas_lat'], main.iloc[candidates.iloc[i]]['gas_long'])
        coords_2 = (main.iloc[candidates.iloc[j]]['gas_lat'], main.iloc[candidates.iloc[j]]['gas_long'])
        dist = distance.distance(coords_1, coords_2).miles
        if dist < 1: 
          if candidates.iloc[i] in adj:
            adj[candidates.iloc[i]].append(candidates.iloc[j])
          else:
            adj[candidates.iloc[i]] = [candidates.iloc[j]]
          if candidates.iloc[j] in adj:
            adj[candidates.iloc[j]].append(candidates.iloc[i])
          else:
            adj[candidates.iloc[j]] = [candidates.iloc[i]]
  return adj

def filter_gas(candidates):
  # remove adjacent gas stations from list, keep only one
  
  adj = get_adjacenct_gas(candidates)
  # print(adj)
  result = dict()
  for i in adj.keys():
    tmp = True
    if i in result: tmp = False
    for j in adj[i]:
      # print(j, j in result)
      if j in result: tmp = False
    if tmp: result[i] = len(adj[i])
  return result

In [ ]:
# adj_sorted = {k: v for k, v in sorted(adj.items(), key=lambda item: len(item[1]))}
# adj_sorted

filtered_gas = filter_gas(df_x['gas_key'])

filtered_df = pd.DataFrame(filtered_gas.values(),index=filtered_gas.keys()).reset_index()
filtered_df = filtered_df.rename(columns={'index': 'gas_key', 0: 'nearby_gas'})
df_x_filtered = filtered_df.merge(df_x, how = 'right', right_on = 'gas_key', left_on = 'gas_key').dropna(subset = ['nearby_gas'])

## Start Optimization

In [ ]:
import gurobipy as gp
from gurobipy import GRB

In [ ]:
# p = 0.008 # 0.02, 0.04, 0.1
def ev_station_loc_optimize(input, p, ratio = 261):
  m = gp.Model()

  # variables
  A = input['gas_key'].tolist()
  # number of EVs nearby 5 miles -> dict
  n = input[['gas_key', 'num_EV_in_5_miles_of_gas']].set_index('gas_key').T.to_dict("index")['num_EV_in_5_miles_of_gas']
  # traffic count -> dict
  T = input[['gas_key', 'traff_cnt_5m_max']].set_index('gas_key').T.to_dict("index")['traff_cnt_5m_max'] # tried with avg

  G = input[['gas_key', 'nearby_gas']].set_index('gas_key').T.to_dict("index")['nearby_gas'] # tried with avg

  # Add gas station locations
  y = m.addVars(A, vtype=GRB.INTEGER, name='gas') #GRB.INTEGER => optimize charger #BINARY

  # Set objective function
  m.setObjective(y.sum(), GRB.MINIMIZE)

  # temporary facilities capacity constraints
  demand_constraints = m.addConstrs((ratio*(n[i]+y[i]) >= T[i]*p for i in A))
  m.optimize()

  result = {}
  # print(f"\n_____________Plan for temporary facilities______________________")
  for i in A:
    if (y[i].x > 0):
      # print(f"Build an EV station at location {i}, {y[i].x}")
      result[i] = y[i].x
  return result


# 4. Plot Results

In [ ]:
import folium

def plot_result(result):
  ev_station = pd.read_csv('Data/wa_EV_stations.csv')
  ev_station = ev_station[['Latitude','Longitude']]
  ev_station['type'] = 'EV'
  ev_station['color'] = 'green'
  ev_station['radius'] = 50
  ev_station['gas_key'] = ''
  plot_df = ev_station


  for i in result.keys():
    new_df = pd.DataFrame([[main.iloc[i]['gas_lat'], main.iloc[i]['gas_long'], 'Proposed', 'blue', 100, str(i)]], columns=plot_df.columns)
    plot_df = pd.concat([plot_df, new_df], ignore_index=True)


    # center = 46.2735210909813, -122.89553326963093
    center = 46.673550, -120.578784

  m = folium.Map(location=center, 
                zoom_start=7,
                width=650,height=650)

  # Same as before... go through each home in set, make circle, and add to map.
  # This time we add a color using price and the colormap object
  for i in range(plot_df.shape[0]):
      folium.Circle(
          location=[plot_df.iloc[i]['Latitude'], plot_df.iloc[i]['Longitude']],
          popup=plot_df.iloc[i]['gas_key'],
          radius=800,
          fill=True,
          color = plot_df.iloc[i]['color'],
          fill_opacity=0.2
      ).add_to(m)

  return m


### p=0.008

In [ ]:
result_dict = ev_station_loc_optimize(df_x_filtered, p=0.008)
print(f"\n\n\nSuggested {len(result_dict)} locations")
m = plot_result(result_dict)
m

### p = 0.02

In [ ]:
result_dict = ev_station_loc_optimize(df_x_filtered, p=0.02)
print(f"\n\n\nSuggested {len(result_dict)} locations")
m = plot_result(result_dict)
m

### p=0.04

In [ ]:
result_dict = ev_station_loc_optimize(df_x_filtered, p=0.04)
print(f"\n\n\nSuggested {len(result_dict)} locations")
m = plot_result(result_dict)
m

### p=0.1

In [ ]:
result_dict = ev_station_loc_optimize(df_x_filtered, p=0.1)
print(f"\n\n\nSuggested {len(result_dict)} locations")
m = plot_result(result_dict)
m

In [ ]:
print(pd.__version__)
print(np.__version__)
print(folium.__version__)
print(gp.gurobi.version())
import geopy as geo
print(geo.__version__)
